In [1]:
import pandas as pd
import numpy as np
import warnings
import multiprocessing
from sklearn.feature_selection import SelectKBest,f_classif,f_regression
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import set_config

In [2]:
warnings.filterwarnings("ignore")

In [3]:
set_config(display='diagram')

In [4]:
random_state = 27912

In [5]:
bbdd=pd.read_csv("../input/bbdd-tfg-aprendizaje/bbdd.csv",sep=",",encoding='utf-8')

In [6]:
bbdd

,Año,Fecha,País,Jornada,Equipo local,Equipo visitante,Últimos partidos local,Últimos partidos visitante,Puntos totales local,Puntos totales visitante,...,Tarjetas rojas equipo visitante,Faltas equipo local,Faltas equipo visitante,Córners equipo local,Córners equipo visitante,Paradas equipo local,Paradas equipo visitante,Resultado,Clustering local,Clustering visitante
0,2015,21/08/2015,ESP,1,Málaga,Sevilla,DDVDD,VDEVV,0.0,0.0,...,1,19,11,7,2,2,4,1,NaN,NaN
1,2015,22/08/2015,ESP,1,Espanyol,Getafe,VEVDD,DDDED,0.0,0.0,...,1,19,14,4,6,2,1,0,NaN,NaN
2,2015,22/08/2015,ESP,1,La Coruña,Real Sociedad,DEEVE,EVDDV,0.0,0.0,...,0,16,10,5,4,2,5,1,NaN,NaN
3,2015,22/08/2015,ESP,1,Atlético Madrid,Las Palmas,VEEDE,VVVVV,0.0,0.0,...,0,11,16,4,4,1,2,0,NaN,NaN
4,2015,22/08/2015,ESP,1,Rayo Vallecano,Valencia,EEVVD,EVEEV,0.0,0.0,...,0,19,11,3,3,4,3,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10850,2020,23/05/2021,ENG,38,West Ham,Southampton,DVDEV,EDVVD,62.0,43.0,...,0,7,14,2,3,5,4,0,2.0,4.0
10851,2020,23/05/2021,ENG,38,Fulham,Newcastle Utd,EDDDE,EDVDV,28.0,42.0,...,0,14,6,5,4,2,0,2,4.0,4.0
10852,2020,23/05/2021,ENG,38,Arsenal,Brighton,DVVVV,DVDEV,58.0,41.0,...,0,10,15,11,3,1,2,0,2.0,2.0
10853,2020,23/05/2021,ENG,38,Sheffield Utd,Burnley,VDDVD,VDVDD,20.0,39.0,...,0,13,3,8,9,3,2,0,0.0,4.0


In [7]:
bbdd_aprendizaje=bbdd.drop(columns=['Año', 'Fecha', 'Marcador', 'Goles equipo local', 'Goles equipo visitante', 'Diferencia de goles', 'Puntos equipo local', 'Puntos equipo visitante'])

In [8]:
bbdd_esp=bbdd_aprendizaje.iloc[0:2280]
bbdd_it=bbdd_aprendizaje.iloc[2280:4560]
bbdd_ale=bbdd_aprendizaje.iloc[4560:6396]
bbdd_fr=bbdd_aprendizaje.iloc[6396:8575]
bbdd_en=bbdd_aprendizaje.iloc[8575:10855]

In [9]:
bbdd_esp_train=bbdd_esp.iloc[0:1520]
bbdd_esp_validation=bbdd_esp.iloc[1520:1900]
bbdd_esp_test=bbdd_esp.iloc[1900:2280]

In [10]:
bbdd_it_train=bbdd_it.iloc[0:1520]
bbdd_it_validation=bbdd_it.iloc[1520:1900]
bbdd_it_test=bbdd_it.iloc[1900:2280]

In [11]:
bbdd_ale_train=bbdd_ale.iloc[0:1224]
bbdd_ale_validation=bbdd_ale.iloc[1224:1530]
bbdd_ale_test=bbdd_ale.iloc[1530:1836]

In [12]:
bbdd_fr_train=bbdd_fr.iloc[0:1520]
bbdd_fr_validation=bbdd_fr.iloc[1520:1799]
bbdd_fr_test=bbdd_fr.iloc[1799:2179]

In [13]:
bbdd_en_train=bbdd_en.iloc[0:1520]
bbdd_en_validation=bbdd_en.iloc[1520:1900]
bbdd_en_test=bbdd_en.iloc[1900:2280]

In [14]:
numeric_features = list(bbdd_aprendizaje.select_dtypes(include=['float64', 'int64']).columns)
numeric_features.remove("Resultado")
categorical_features = ["País", "Equipo local", "Equipo visitante", "Últimos partidos local", "Últimos partidos visitante"]

In [15]:
trf1 = ColumnTransformer(transformers =[ 
    ('cat', SimpleImputer(strategy ='most_frequent'), categorical_features), 
    ('num', KNNImputer(n_neighbors=3), numeric_features), 
], remainder ='passthrough') 

In [16]:
trf2 = ColumnTransformer(transformers =[ 
    ('enc', OneHotEncoder(handle_unknown="ignore", sparse = False), list(range(5))), 
], remainder ='passthrough') 

In [17]:
naive_bayes_model = GaussianNB()

In [18]:
pipeline_nvm = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('NVM', naive_bayes_model),
])
pipeline_nvm

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'Tarjetas rojas equipo '
                                                   'local',
                                                   'Tarjetas rojas equipo '
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('NVM', GaussianNB())])

In [19]:
decision_tree_model = DecisionTreeClassifier(random_state=random_state)

In [20]:
pipeline_dtm = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('DTM', decision_tree_model),
])
pipeline_dtm

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'Tarjetas rojas equipo '
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('DTM', DecisionTreeClassifier(random_state=27912))])

In [21]:
adaboost_model = AdaBoostClassifier(random_state=random_state)

In [22]:
pipeline_adaboost = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('adaboost', adaboost_model),
])
pipeline_adaboost

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('adaboost', AdaBoostClassifier(random_state=27912))])

In [23]:
bagging_model = BaggingClassifier(random_state=random_state)

In [24]:
pipeline_bagging = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('bagging', bagging_model),
])
pipeline_bagging

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'Tarjetas rojas equipo '
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('bagging', BaggingClassifier(random_state=27912))])

In [25]:
random_forest_model = RandomForestClassifier(random_state=random_state)

In [26]:
pipeline_rf = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('rf', random_forest_model),
])
pipeline_rf

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'Tarjetas rojas equipo '
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('rf', RandomForestClassifier(random_state=27912))])

In [27]:
gradient_boosting_model = GradientBoostingClassifier(random_state=random_state)

In [28]:
pipeline_gtb = Pipeline(steps =[ 
    ('tf1', trf1), 
    ('tf2', trf2), 
    ('kbest', SelectKBest(f_regression)),
    ('gtb', gradient_boosting_model),
])
pipeline_gtb

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['País', 'Equipo local',
                                                   'Equipo visitante',
                                                   'Últimos partidos local',
                                                   'Últimos partidos '
                                                   'visitante']),
                                                 ('num',
                                                  KNNImputer(n_neighbors=3),
                                                  ['Jornada',
                                                   'Puntos totales local',
                                                   'Puntos totales visitante',
                                                   'PG local', 'PG visitante',
                                                   'PE local', 'PE vis...
                                                   'visitante',
                                                   'Faltas equipo local',
                                                   'Faltas equipo visitante',
                                                   'Córners equipo local', ...])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('enc',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [0, 1, 2, 3, 4])])),
                ('kbest',
                 SelectKBest(score_func=<function f_regression at 0x7f57d52805f0>)),
                ('gtb', GradientBoostingClassifier(random_state=27912))])

In [29]:
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=random_state)

In [30]:
def optimize_params(estimator, X, y, cv, scoring=None, refit=True, **param_grid):
    """Exhaustive search over specified parameter values for an estimator."""
    grid_search_cv = GridSearchCV(estimator,
                                  param_grid,
                                  scoring=scoring,
                                  refit=refit,
                                  cv=cv,
                                  return_train_score=True).fit(X, y)

    cv_results = pd.DataFrame(grid_search_cv.cv_results_)

    # Drop the results for each validation split and sort by the refit metric
    labels = cv_results.filter(regex="split")
    by = cv_results.filter(regex="rank_test").columns[0]
    cv_results = cv_results.drop(labels, axis=1).sort_values(by)

    display(cv_results)
    
    return grid_search_cv

def evaluate_estimators(estimators, X, y, *metrics):
    """Evaluate the estimators using the specified metrics."""
    results = pd.DataFrame(columns=metrics)

    for metric in metrics:
        for estimator in estimators:
            # Set the index of the results to the estimator class name
            name = estimator.estimator[1].__class__.__name__

            # Determine the scorer for evaluating the estimator
            scorer = check_scoring(estimator, metric)

            results.loc[name, metric] = scorer(estimator, X, y)

    return results

In [31]:
naive_bayes_classifier = optimize_params(pipeline_nvm, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kbest__k,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
1,0.137879,0.002335,0.094471,0.002035,11,{'kbest__k': 11},0.489868,0.035080,1,0.513684,0.034217
0,0.139429,0.005705,0.094527,0.004431,10,{'kbest__k': 10},0.486316,0.038050,2,0.506579,0.041787
2,0.138789,0.002416,0.097911,0.010538,12,{'kbest__k': 12},0.484342,0.034379,3,0.506316,0.033787
3,0.138933,0.004647,0.094121,0.001442,13,{'kbest__k': 13},0.482895,0.032203,4,0.505526,0.038098
9,0.139047,0.002225,0.095744,0.002399,19,{'kbest__k': 19},0.482368,0.030854,5,0.504342,0.033459
7,0.159828,0.022958,0.110201,0.020503,17,{'kbest__k': 17},0.477500,0.026134,6,0.499868,0.033076
8,0.137563,0.001937,0.094630,0.001549,18,{'kbest__k': 18},0.475000,0.028428,7,0.499342,0.032706
5,0.138183,0.001804,0.094445,0.000967,15,{'kbest__k': 15},0.474605,0.023242,8,0.498289,0.032241
4,0.138084,0.002607,0.094883,0.002037,14,{'kbest__k': 14},0.474342,0.024842,9,0.502105,0.038118
6,0.139859,0.002764,0.095214,0.001495,16,{'kbest__k': 16},0.469211,0.028883,10,0.491053,0.039455


In [32]:
criterion = ["gini", "entropy"]

In [33]:
max_depth = [2, 3, 5, None]

In [34]:
decision_tree_classifier = optimize_params(pipeline_dtm, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, DTM__criterion=criterion, DTM__max_depth=max_depth,kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_DTM__criterion,param_DTM__max_depth,param_kbest__k,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
19,0.141982,0.002665,0.096261,0.000889,gini,3,19,"{'DTM__criterion': 'gini', 'DTM__max_depth': 3...",0.498421,0.010751,1,0.537237,0.019886
17,0.152895,0.026091,0.100876,0.017180,gini,3,17,"{'DTM__criterion': 'gini', 'DTM__max_depth': 3...",0.495789,0.014709,2,0.535263,0.023061
18,0.141302,0.002878,0.095190,0.002633,gini,3,18,"{'DTM__criterion': 'gini', 'DTM__max_depth': 3...",0.495789,0.014638,3,0.535132,0.023047
16,0.139118,0.001596,0.095953,0.006632,gini,3,16,"{'DTM__criterion': 'gini', 'DTM__max_depth': 3...",0.494474,0.013557,4,0.533421,0.022769
15,0.140647,0.005435,0.096070,0.006804,gini,3,15,"{'DTM__criterion': 'gini', 'DTM__max_depth': 3...",0.493816,0.013560,5,0.531711,0.023927
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,0.141302,0.001497,0.094640,0.002666,gini,None,10,"{'DTM__criterion': 'gini', 'DTM__max_depth': N...",0.397500,0.023826,76,0.998684,0.001557
31,0.142624,0.001737,0.093963,0.001010,gini,None,11,"{'DTM__criterion': 'gini', 'DTM__max_depth': N...",0.397105,0.016900,77,0.998684,0.001557
73,0.148476,0.002316,0.095442,0.001832,entropy,None,13,"{'DTM__criterion': 'entropy', 'DTM__max_depth'...",0.396711,0.015314,78,0.999079,0.001562
32,0.143376,0.002367,0.095048,0.002256,gini,None,12,"{'DTM__criterion': 'gini', 'DTM__max_depth': N...",0.394079,0.022883,79,0.999079,0.001562


In [35]:
base_estimator = [DecisionTreeClassifier(random_state=random_state)]

In [36]:
n_estimators = [20, 50, 100]

In [37]:
learning_rate = [0.95, 1.0]

In [38]:
max_depth = [2, 3]

In [39]:
adaboost_classifier = optimize_params(pipeline_adaboost, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, adaboost__base_estimator=base_estimator, adaboost__n_estimators=n_estimators, adaboost__learning_rate=learning_rate, adaboost__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_adaboost__base_estimator,param_adaboost__base_estimator__max_depth,param_adaboost__learning_rate,param_adaboost__n_estimators,param_kbest__k,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
2,0.234289,0.002976,0.112097,0.001610,"DecisionTreeClassifier(max_depth=2, random_sta...",2,0.95,20,12,{'adaboost__base_estimator': DecisionTreeClass...,0.479868,0.012594,1,0.603026,0.013750
37,0.244235,0.003201,0.118276,0.001869,"DecisionTreeClassifier(max_depth=2, random_sta...",2,1.0,20,17,{'adaboost__base_estimator': DecisionTreeClass...,0.471974,0.018692,2,0.617105,0.010850
32,0.233404,0.003699,0.112000,0.002011,"DecisionTreeClassifier(max_depth=2, random_sta...",2,1.0,20,12,{'adaboost__base_estimator': DecisionTreeClass...,0.471711,0.011153,3,0.603816,0.015134
35,0.248010,0.024230,0.117137,0.008896,"DecisionTreeClassifier(max_depth=2, random_sta...",2,1.0,20,15,{'adaboost__base_estimator': DecisionTreeClass...,0.470921,0.016449,4,0.612895,0.007462
39,0.245955,0.005081,0.120171,0.000753,"DecisionTreeClassifier(max_depth=2, random_sta...",2,1.0,20,19,{'adaboost__base_estimator': DecisionTreeClass...,0.470132,0.019868,5,0.621447,0.015255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,0.432075,0.010930,0.160330,0.001522,"DecisionTreeClassifier(max_depth=2, random_sta...",3,1.0,100,10,{'adaboost__base_estimator': DecisionTreeClass...,0.415395,0.011258,116,0.860263,0.013299
20,0.415129,0.021056,0.164812,0.012476,"DecisionTreeClassifier(max_depth=2, random_sta...",2,0.95,100,10,{'adaboost__base_estimator': DecisionTreeClass...,0.415132,0.013396,117,0.727500,0.015740
81,0.444278,0.003684,0.163620,0.001012,"DecisionTreeClassifier(max_depth=2, random_sta...",3,0.95,100,11,{'adaboost__base_estimator': DecisionTreeClass...,0.414474,0.013585,118,0.866711,0.019938
71,0.308204,0.004240,0.136076,0.001034,"DecisionTreeClassifier(max_depth=2, random_sta...",3,0.95,50,11,{'adaboost__base_estimator': DecisionTreeClass...,0.413289,0.014250,119,0.807237,0.019554


In [40]:
bagging_classifier = optimize_params(pipeline_bagging, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, bagging__base_estimator=base_estimator, bagging__n_estimators=n_estimators, bagging__base_estimator__max_depth=max_depth,kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_bagging__base_estimator,param_bagging__base_estimator__max_depth,param_bagging__n_estimators,param_kbest__k,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
58,0.467714,0.011138,0.180934,0.003076,"DecisionTreeClassifier(max_depth=3, random_sta...",3,100,18,{'bagging__base_estimator': DecisionTreeClassi...,0.511842,0.009934,1,0.561184,0.012569
39,0.241584,0.009612,0.118793,0.002747,"DecisionTreeClassifier(max_depth=3, random_sta...",3,20,19,{'bagging__base_estimator': DecisionTreeClassi...,0.511184,0.008910,2,0.556711,0.012654
59,0.476708,0.008738,0.185016,0.002248,"DecisionTreeClassifier(max_depth=3, random_sta...",3,100,19,{'bagging__base_estimator': DecisionTreeClassi...,0.511053,0.009779,3,0.560132,0.012483
49,0.325183,0.004030,0.146002,0.001364,"DecisionTreeClassifier(max_depth=3, random_sta...",3,50,19,{'bagging__base_estimator': DecisionTreeClassi...,0.510789,0.009300,4,0.558158,0.012028
44,0.309163,0.006251,0.139247,0.001312,"DecisionTreeClassifier(max_depth=3, random_sta...",3,50,14,{'bagging__base_estimator': DecisionTreeClassi...,0.510789,0.010654,5,0.555000,0.010895
38,0.235608,0.002355,0.116786,0.001818,"DecisionTreeClassifier(max_depth=3, random_sta...",3,20,18,{'bagging__base_estimator': DecisionTreeClassi...,0.510658,0.008206,6,0.555789,0.013588
48,0.323162,0.007829,0.145136,0.000980,"DecisionTreeClassifier(max_depth=3, random_sta...",3,50,18,{'bagging__base_estimator': DecisionTreeClassi...,0.510395,0.009551,7,0.558026,0.011016
57,0.463114,0.013116,0.180201,0.004274,"DecisionTreeClassifier(max_depth=3, random_sta...",3,100,17,{'bagging__base_estimator': DecisionTreeClassi...,0.510263,0.009074,8,0.560000,0.011874
46,0.315009,0.001907,0.143107,0.004393,"DecisionTreeClassifier(max_depth=3, random_sta...",3,50,16,{'bagging__base_estimator': DecisionTreeClassi...,0.509868,0.010398,9,0.556316,0.011648
55,0.436481,0.007933,0.169298,0.000946,"DecisionTreeClassifier(max_depth=3, random_sta...",3,100,15,{'bagging__base_estimator': DecisionTreeClassi...,0.509737,0.010949,10,0.560395,0.012008


In [41]:
max_features = ["sqrt", "log2"]

In [42]:
random_forest_classifier = optimize_params(pipeline_rf, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, rf__n_estimators=n_estimators, rf__criterion=criterion, rf__max_features=max_features, kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kbest__k,param_rf__criterion,param_rf__max_features,param_rf__n_estimators,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
113,0.390484,0.006499,0.129411,0.001200,19,gini,log2,100,"{'kbest__k': 19, 'rf__criterion': 'gini', 'rf_...",0.496842,0.009762,1,1.000000,0.000000
110,0.391091,0.005746,0.128645,0.001510,19,gini,sqrt,100,"{'kbest__k': 19, 'rf__criterion': 'gini', 'rf_...",0.496842,0.009762,1,1.000000,0.000000
107,0.420128,0.010111,0.128254,0.001913,18,entropy,log2,100,"{'kbest__k': 18, 'rf__criterion': 'entropy', '...",0.493289,0.011613,3,1.000000,0.000000
104,0.416427,0.007407,0.130300,0.004662,18,entropy,sqrt,100,"{'kbest__k': 18, 'rf__criterion': 'entropy', '...",0.493289,0.011613,3,1.000000,0.000000
98,0.392080,0.009996,0.127174,0.004243,18,gini,sqrt,100,"{'kbest__k': 18, 'rf__criterion': 'gini', 'rf_...",0.492632,0.014428,5,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,0.217124,0.004496,0.102427,0.001979,11,gini,log2,20,"{'kbest__k': 11, 'rf__criterion': 'gini', 'rf_...",0.458684,0.012800,115,0.995789,0.002551
4,0.279701,0.002400,0.107570,0.002050,10,gini,log2,50,"{'kbest__k': 10, 'rf__criterion': 'gini', 'rf_...",0.457237,0.012163,117,0.998684,0.001557
1,0.293912,0.028969,0.114733,0.016845,10,gini,sqrt,50,"{'kbest__k': 10, 'rf__criterion': 'gini', 'rf_...",0.457237,0.012163,117,0.998684,0.001557
3,0.211967,0.005732,0.101153,0.002607,10,gini,log2,20,"{'kbest__k': 10, 'rf__criterion': 'gini', 'rf_...",0.455395,0.011613,119,0.995789,0.001842


In [43]:
learning_rate = [0.01, 0.05, 0.1]

In [44]:
criterion = ["friedman_mse", "squared_error"]

In [45]:
gradient_boosting_classifier = optimize_params(pipeline_gtb, bbdd_esp_train.drop(columns=["Resultado"]), bbdd_esp_train["Resultado"], cv, gtb__learning_rate=learning_rate, gtb__n_estimators=n_estimators, gtb__criterion=criterion, gtb__max_depth=max_depth, kbest__k=range(10,20))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_gtb__criterion,param_gtb__learning_rate,param_gtb__max_depth,param_gtb__n_estimators,param_kbest__k,params,mean_test_score,std_test_score,rank_test_score,mean_train_score,std_train_score
88,0.598520,0.014830,0.097074,0.001598,friedman_mse,0.05,2,100,18,"{'gtb__criterion': 'friedman_mse', 'gtb__learn...",0.519868,0.010664,1,0.610000,0.009708
268,0.609491,0.020908,0.098666,0.001570,squared_error,0.05,2,100,18,"{'gtb__criterion': 'squared_error', 'gtb__lear...",0.519737,0.010738,2,0.610000,0.009708
89,0.622061,0.021843,0.098917,0.001776,friedman_mse,0.05,2,100,19,"{'gtb__criterion': 'friedman_mse', 'gtb__learn...",0.518421,0.012230,3,0.609079,0.008163
269,0.608762,0.018679,0.098318,0.001355,squared_error,0.05,2,100,19,"{'gtb__criterion': 'squared_error', 'gtb__lear...",0.518421,0.012230,3,0.609079,0.008163
85,0.570843,0.030588,0.097511,0.002393,friedman_mse,0.05,2,100,15,"{'gtb__criterion': 'friedman_mse', 'gtb__learn...",0.516447,0.011414,5,0.605000,0.010391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,0.257187,0.002943,0.094265,0.007459,friedman_mse,0.01,2,20,16,"{'gtb__criterion': 'friedman_mse', 'gtb__learn...",0.467763,0.000658,345,0.467763,0.000658
7,0.258896,0.002308,0.092395,0.001581,friedman_mse,0.01,2,20,17,"{'gtb__criterion': 'friedman_mse', 'gtb__learn...",0.467763,0.000658,345,0.467763,0.000658
186,0.261288,0.005793,0.098112,0.011650,squared_error,0.01,2,20,16,"{'gtb__criterion': 'squared_error', 'gtb__lear...",0.467763,0.000658,345,0.467763,0.000658
188,0.266613,0.007663,0.097031,0.010369,squared_error,0.01,2,20,18,"{'gtb__criterion': 'squared_error', 'gtb__lear...",0.467763,0.000658,345,0.467763,0.000658
